In [ ]:
import sunpy.map

import matplotlib.pyplot as plt

from sdoml import SDOMLDataset
from timeit import default_timer as timer

First, we will instantiate the ``SDOMLDataset`` class, to load one month of 
the six optically-thin SDO/AIA channels (94A/131A/171A/193A/211A/335A) alongside the 3 components of the HMI magnetograms (Bx, By, Bz)
from ``fdl-sdoml-v2/sdomlv2_small.zarr`` and ``fdl-sdoml-v2/sdomlv2_hmi_small.zarr/``

In [ ]:
sdomlds = SDOMLDataset(
    storage_location="gcs",
    zarr_root={"AIA": "fdl-sdoml-v2/sdomlv2_small.zarr/",
               "HMI": "fdl-sdoml-v2/sdomlv2_hmi_small.zarr/"},
    cache_max_size=1 * 512 * 512 * 4096,
    years=["2010"],
    channels = {
        "AIA": ["94A", "131A", "171A", "193A", "211A", "335A"],
        "HMI": ["Bx", "By", "Bz"]
    }
)

With the Dataset instantiated, we will directly access the dataset using the ``__getitem__`` method. ``sdomlds.__getitem__(idx)`` loads and returns a sample from the dataset at the given index ``idx``.  

As will be evident, the first data access for this index is relatively slow (it is retrieved from remote store on Google Cloud Storage), however the second data access is faster, as this uses cache. For more information see https://zarr.readthedocs.io/en/stable/api/storage.html#zarr.storage.LRUStoreCache

In [ ]:
# obtain the first item of the dataset
idx = 0

# extract image and metadata:
images, metadata = sdomlds.__getitem__(idx)

``SDOMLDataset()`` returns both image, and metadata. 

For the example here, the full image dataset is of size ``(6133, 9, 512, 512)``, where there are 6133 co-temporal observations, each with 9 channels of ``(512, 512)``.

* The ``images`` returned by ``__getitem__(idx)`` are of size: ``(1, 9, 512, 512)``, where each of the 6133 items consist of 6 co-temporal observations (SDO/AIA ``[94, 131, 171, 193, 211, 335]``); SDO/HMI ``[Bx, By, Bz]`` of ``torch.Size([512, 512])``. 


* The ``metadata`` is a dictionary of 281 key-value pairs (the union of both AIA (175) and HMI (147)). Each keys value is a list of 9 elements (which corresponds to the 9 channels in the image data; some keys have data associated with HMI, and others AIA). A small excerpt of this dictionary is shown below for ``['DEG_COR', 'EXPTIME', 'WAVELNTH', 'T_OBS']``, where ``DEG_COR`` and ``EXPTIME`` are SDO/AIA keys, and ``WAVELNTH`` and ``T_OBS`` are present for both instruments. A value that does not exist is entered as ``pandas._libs.missing.NAType``


```
{
    'DEG_COR': [1.083, 0.950019, 0.99739, 0.99217, 0.982774, 0.901734, <NA>, <NA>, <NA>],
    'EXPTIME': [2.901124, 2.901351, 1.999653, 2.000068, 2.900861, 2.900854, <NA>, <NA>, <NA>],
    'WAVELNTH': [94, 131, 171, 193, 211, 335, 6173.0, 6173.0, 6173.0],
    'T_OBS':
       ['2010-08-01T00:00:09.57Z',
        '2010-08-01T00:00:11.07Z',
        '2010-08-01T00:00:01.34Z',
        '2010-08-01T00:00:08.84Z',
        '2010-08-01T00:00:02.07Z',
        '2010-08-01T00:00:05.07Z',
        '2010.08.01_00:00:07_TAI',
        '2010.08.01_00:00:07_TAI',
        '2010.08.01_00:00:07_TAI']
}
```

### Plotting one set of images

For the one set of images returned from the dataloader, the following code block creates the set of ``sunpy.map`` from the ``images`` and ``metadata``.

In [ ]:
plt.figure(figsize=(20, 20))

for img_index in range(images[0,:,0,0].shape[0]):
    # Create a sunpy map with the data
    selected_image = images[0,img_index,:,:]
    selected_headr = {keys: values[img_index] for keys, values in metadata.items()}
    my_map = sunpy.map.Map(selected_image.numpy(), selected_headr)
    
    # set the index and plot the sunpy.map
    ax = plt.subplot(3, 3, img_index+1, projection=my_map)
    
    if img_index >= 6:
        my_map.plot_settings['cmap'] = "hmimag"
        my_map.plot_settings['norm'] = plt.Normalize(-1000.,1000.)
        
    my_map.plot()